# K线图基本构成

## 📋 概述

K线图是技术分析的基础工具，通过图形化的方式展示股票价格的变化。理解K线图的基本构成是学习技术分析的第一步。本章节将详细介绍K线图的构成要素、绘制方法和阅读技巧。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际操作加深理解。


## 📦 环境准备

首先安装必要的Python库：


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib mplfinance akshare

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import akshare as ak
from datetime import datetime, timedelta

# 设置中文字体（解决中文显示问题）
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. K线的四个价格要素

K线由四个价格要素构成：
- **开盘价（Open）**：交易日第一笔成交的价格，用O表示
- **收盘价（Close）**：交易日最后一笔成交的价格，用C表示
- **最高价（High）**：交易日最高成交价格，用H表示
- **最低价（Low）**：交易日最低成交价格，用L表示

让我们通过实际数据来理解这些概念：


In [ ]:
# 获取股票数据（以平安银行000001为例）
# 注意：这里使用AKShare获取数据，如果网络有问题可能需要梯子
try:
    # 获取最近30天的数据
    stock_data = ak.stock_zh_a_hist(symbol="000001", period="daily", 
                                     start_date="20240101", end_date="20241231", 
                                     adjust="qfq")
    
    # 重命名列名，使其符合标准OHLC格式
    stock_data.columns = ['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率']
    stock_data['日期'] = pd.to_datetime(stock_data['日期'])
    stock_data = stock_data.set_index('日期')
    stock_data = stock_data[['开盘', '最高', '最低', '收盘', '成交量']]
    stock_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    
    print("数据获取成功！")
    print(f"\n数据形状：{stock_data.shape}")
    print(f"\n最近5天的数据：")
    print(stock_data.tail())
    
except Exception as e:
    print(f"数据获取失败：{e}")
    print("使用模拟数据演示...")
    # 创建模拟数据
    dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
    dates = dates[dates.weekday < 5]  # 只保留工作日
    
    np.random.seed(42)
    prices = 10 + np.cumsum(np.random.randn(len(dates)) * 0.1)
    
    stock_data = pd.DataFrame({
        'Open': prices + np.random.randn(len(dates)) * 0.05,
        'High': prices + np.abs(np.random.randn(len(dates)) * 0.1),
        'Low': prices - np.abs(np.random.randn(len(dates)) * 0.1),
        'Close': prices + np.random.randn(len(dates)) * 0.05,
        'Volume': np.random.randint(1000000, 10000000, len(dates))
    }, index=dates)
    
    # 确保High >= max(Open, Close), Low <= min(Open, Close)
    stock_data['High'] = stock_data[['Open', 'Close', 'High']].max(axis=1)
    stock_data['Low'] = stock_data[['Open', 'Close', 'Low']].min(axis=1)
    
    print("使用模拟数据演示")
    print(f"\n数据形状：{stock_data.shape}")
    print(f"\n最近5天的数据：")
    print(stock_data.tail())


In [ ]:
# 使用mplfinance绘制K线图（最近60天数据）
recent_data = stock_data.tail(60)

# 绘制K线图
mpf.plot(recent_data, 
         type='candle',  # 蜡烛图
         style='charles',  # 样式
         volume=True,  # 显示成交量
         title='K线图示例 - 平安银行(000001)',
         ylabel='价格',
         ylabel_lower='成交量',
         figsize=(15, 8),
         show_nontrading=False)


### 3. 不同周期的K线图

K线图可以按不同周期绘制：
- **日K线**：每根K线代表一个交易日
- **周K线**：每根K线代表一周
- **月K线**：每根K线代表一个月
- **分钟K线**：1分钟、5分钟、15分钟等

让我们看看不同周期的K线图：


In [ ]:
# 将日K线数据转换为周K线和月K线
# 周K线：取每周最后一个交易日的数据
weekly_data = stock_data.resample('W').agg({
    'Open': 'first',  # 周开盘价 = 周一开盘价
    'High': 'max',    # 周最高价 = 一周最高价
    'Low': 'min',     # 周最低价 = 一周最低价
    'Close': 'last',  # 周收盘价 = 周五收盘价
    'Volume': 'sum'   # 周成交量 = 一周成交量之和
})

# 月K线：取每月最后一个交易日的数据
monthly_data = stock_data.resample('M').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})

# 绘制对比图
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 日K线
mpf.plot(stock_data.tail(60), type='candle', style='charles', 
         ax=axes[0], volume=axes[0])
axes[0].set_title('日K线图（最近60天）', fontsize=14, fontweight='bold')

# 周K线
mpf.plot(weekly_data.tail(20), type='candle', style='charles',
         ax=axes[1], volume=axes[1])
axes[1].set_title('周K线图（最近20周）', fontsize=14, fontweight='bold')

# 月K线
mpf.plot(monthly_data.tail(12), type='candle', style='charles',
         ax=axes[2], volume=axes[2])
axes[2].set_title('月K线图（最近12个月）', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("不同周期的K线图对比完成！")
print(f"日K线数据点：{len(stock_data.tail(60))}")
print(f"周K线数据点：{len(weekly_data.tail(20))}")
print(f"月K线数据点：{len(monthly_data.tail(12))}")


### 4. 常见K线形态识别

让我们识别一些常见的K线形态：


In [ ]:
# 识别常见的K线形态
def identify_candle_pattern(data):
    """识别K线形态"""
    patterns = []
    
    for i in range(len(data)):
        row = data.iloc[i]
        open_price = row['Open']
        close_price = row['Close']
        high_price = row['High']
        low_price = row['Low']
        
        body = abs(close_price - open_price)
        upper_shadow = high_price - max(open_price, close_price)
        lower_shadow = min(open_price, close_price) - low_price
        total_range = high_price - low_price
        
        # 判断K线类型
        if body < total_range * 0.1:  # 实体很小
            if upper_shadow > lower_shadow * 2:
                patterns.append('倒T字线')
            elif lower_shadow > upper_shadow * 2:
                patterns.append('T字线')
            else:
                patterns.append('十字星')
        elif close_price > open_price:  # 阳线
            if body > total_range * 0.7:
                patterns.append('大阳线')
            elif lower_shadow > body * 2:
                patterns.append('锤子线')
            else:
                patterns.append('普通阳线')
        else:  # 阴线
            if body > total_range * 0.7:
                patterns.append('大阴线')
            elif upper_shadow > body * 2:
                patterns.append('倒锤子线')
            else:
                patterns.append('普通阴线')
    
    return patterns

# 识别最近20天的K线形态
recent_20 = stock_data.tail(20)
patterns = identify_candle_pattern(recent_20)

# 创建结果DataFrame
pattern_df = pd.DataFrame({
    '日期': recent_20.index,
    '开盘': recent_20['Open'].round(2),
    '收盘': recent_20['Close'].round(2),
    '最高': recent_20['High'].round(2),
    '最低': recent_20['Low'].round(2),
    'K线形态': patterns
})

print("最近20天的K线形态识别：")
print(pattern_df.to_string(index=False))

# 统计各种形态的数量
pattern_counts = pd.Series(patterns).value_counts()
print(f"\n各种K线形态统计：")
print(pattern_counts)


In [ ]:
# 分析价量关系
recent_data = stock_data.tail(30).copy()

# 计算价格变化和成交量变化
recent_data['价格变化'] = recent_data['Close'].pct_change()
recent_data['成交量变化'] = recent_data['Volume'].pct_change()
recent_data['价量关系'] = ''

# 判断价量关系
for i in range(1, len(recent_data)):
    price_change = recent_data.iloc[i]['价格变化']
    volume_change = recent_data.iloc[i]['成交量变化']
    
    if price_change > 0 and volume_change > 0:
        recent_data.iloc[i, recent_data.columns.get_loc('价量关系')] = '价涨量增 ✅'
    elif price_change > 0 and volume_change < 0:
        recent_data.iloc[i, recent_data.columns.get_loc('价量关系')] = '价涨量缩 ⚠️'
    elif price_change < 0 and volume_change > 0:
        recent_data.iloc[i, recent_data.columns.get_loc('价量关系')] = '价跌量增 ⚠️'
    elif price_change < 0 and volume_change < 0:
        recent_data.iloc[i, recent_data.columns.get_loc('价量关系')] = '价跌量缩 ✅'
    else:
        recent_data.iloc[i, recent_data.columns.get_loc('价量关系')] = '价量平衡'

# 显示价量关系分析
print("价量关系分析（最近30天）：")
print(recent_data[['Close', 'Volume', '价格变化', '成交量变化', '价量关系']].tail(10).to_string())

# 绘制价量关系图
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# 价格图
ax1.plot(recent_data.index, recent_data['Close'], label='收盘价', linewidth=2)
ax1.set_ylabel('价格', fontsize=12)
ax1.set_title('价格与成交量关系分析', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 成交量图
ax2.bar(recent_data.index, recent_data['Volume'], alpha=0.6, color='blue', label='成交量')
ax2.set_xlabel('日期', fontsize=12)
ax2.set_ylabel('成交量', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 💡 关键要点总结

1. **K线由四个价格要素构成**：开盘价、收盘价、最高价、最低价
2. **K线包括实体和影线**：实体反映涨跌，影线反映压力和支撑
3. **不同周期的K线**：日K线、周K线、月K线、分钟K线，适用于不同分析需求
4. **K线分析要结合成交量**：价量配合才能准确判断趋势
5. **K线形态有单根和组合**：不同形态代表不同的市场含义

## 🛠️ 工具与软件

### Python绘图工具

本Notebook使用的工具：
- **mplfinance**：专业K线图绘制库
- **matplotlib**：基础绘图库
- **pandas**：数据处理库
- **AKShare**：数据获取库

### 数据获取

- **AKShare**：支持A股、港股、美股数据，完全免费，无需梯子
- **Tushare**：A股数据，需要注册，免费版有数据限制
- **yfinance**：全球股票数据，可能需要梯子

### 注意事项

- ✅ **代码可运行**：所有代码都可以直接运行
- ✅ **数据获取**：如果AKShare获取失败，代码会自动使用模拟数据
- ✅ **网络要求**：AKShare通常不需要梯子，yfinance可能需要
- ⚠️ **数据延迟**：免费数据可能有延迟

## 🔗 相关知识点

- [K线形态识别](./K线形态识别.md)
- [趋势指标](../02_技术指标/趋势指标.md)
- [图表形态](../03_图表形态/README.md)

## 📚 拓展阅读

- 《日本蜡烛图技术》- K线图经典教材
- 《股票技术分析》- 深入学习技术分析
- 各交易软件帮助文档 - 了解软件使用方法
